In [19]:
import math
from scipy.integrate import dblquad

In [81]:
def semicircle(x, r):
    return (r**2 - x**2)**.5

In [88]:
def const2D(x, y, a):
    return a

In [47]:
# 2D gaussian model
# defined based on http://cxc.cfa.harvard.edu/sherpa/ahelp/gauss2d.html

def x_new(x, y, xpos, ypos, theta):
    return (x - xpos) * math.cos(theta) + (y - ypos) * math.sin(theta)

def y_new(x, y, xpos, ypos, theta):
    return (y - ypos) * math.cos(theta) - (x - xpos) * math.sin(theta)

def r(x, y, xpos, ypos, ellip, theta):
    return (x_new(x, y, xpos, ypos, theta)**2 * (1 - ellip)**2 + y_new(x, y, xpos, ypos, theta)**2)**.5 / (1 - ellip)

def gauss2D(x, y, xpos, ypos, ampl, fwhm, ellip, theta):
    return ampl * math.exp(-2.7725887 * (r(x, y, xpos, ypos, ellip, theta) / fwhm)**2)

In [108]:
rmax = 90 / 2.5

# numbers based on a frozen donut-fitted chxe, position constrained pwn, and unconstrained blob (combinedfit6)
chxeargs = (500 - 499.905, 500 - 499.013, 8.23415e-7, 51.2412, .493759, -.9948)
pwnargs = (500 - 498.111, 500 - 501.868, .000114459, 1, 0, 0)
blobargs = (500 - 507.615, 500 - 500.355, 6.33121e-5, 7.02574, .935119, -1.0454)
bgargs = (7.55887e-7,)

chxevol = dblquad(gauss2D, -rmax, rmax, 
                  lambda x: -semicircle(x, rmax), lambda x: semicircle(x, rmax), 
                  args = chxeargs)[0]

pwnvol = dblquad(gauss2D, -rmax, rmax, 
                 lambda x: -semicircle(x, rmax), lambda x: semicircle(x, rmax), 
                 args = pwnargs)[0]

blobvol = dblquad(gauss2D, -rmax, rmax, 
                  lambda x: -semicircle(x, rmax), lambda x: semicircle(x, rmax), 
                  args = blobargs)[0]

bgvol = dblquad(const2D, -rmax, rmax, 
                lambda x: -semicircle(x, rmax), lambda x: semicircle(x, rmax), 
                args = bgargs)[0]

In [107]:
totalvol = chxevol + pwnvol + blobvol + bgvol

print(chxevol / totalvol)
print(pwnvol / totalvol)
print(blobvol / totalvol)
print(bgvol / totalvol)

0.308708790827
0.128502384078
0.223971151155
0.33881767394
